In [4]:
# adapted from https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
# https://www.youtube.com/watch?v=MqugtGD605k
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector, Dense
import numpy as np
import wandb
from wandb.keras import WandbCallback
import random

wandb.init()
config = wandb.config

class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

# Parameters for the model and dataset.
config.hidden_size = 128
config.batch_size = 128

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
maxlen = 30
output_len = 10
# All the numbers, plus sign and space for padding.
chars = '0123456789+$-,/.ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuwxyv '
ctable = CharacterTable(chars)


import pandas as pd
df = pd.read_csv("sale_prices.csv")
df = df.fillna(0)



raw_prices = df["Price on site"].tolist()
parced_prices = []

int_min_price = [int(price) for price in df["Min price"]]
int_max_price = [int(price) for price in df["Max price"]]
for idx in range(len(int_min_price)):
    toappend = str(int_min_price[idx])
    toappend+=' '*(output_len - len(str(int_min_price[idx])))
    parced_prices.append(toappend)
    
#augmentation (av)
# for i in range(4):
#     for idx in range(len(int_min_price)):
#         raw_price = str(raw_prices[idx])
#         parsed_price = str(int_min_price[idx])
#         num_of_replacements = random.randint(1,5)
#         for i in range(num_of_replacements):
#             to_replace = str(random.randint(1,9))
#             replace_with = str(random.randint(1,9))
#             parsed_price.replace(to_replace,replace_with)
#             raw_price.replace(to_replace,replace_with)

#         parsed_price+=' '*(output_len - len(str(int_min_price[idx])))
#         parced_prices.append(parsed_price)
#         raw_prices.append(raw_price)

#augmentation v2

for i in range(4):
    for idx in range(len(int_min_price)):
        raw_price = str(raw_prices[idx])
        parsed_price = str(int_min_price[idx])
        num_of_replacements = random.randint(1,5)
        for i in range(len(parsed_price)):
            to_replace = parsed_price[i]
            replace_with = str(random.randint(0,9))
            parsed_price.replace(to_replace,replace_with)
            raw_price.replace(to_replace,replace_with)

        parsed_price+=' '*(output_len - len(str(int_min_price[idx])))
        parced_prices.append(parsed_price)
        raw_prices.append(raw_price)

print('Total prices:', len(raw_prices))

print('Vectorization...')
x = np.zeros((len(raw_prices), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(raw_prices), output_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(raw_prices):
    x[i] = ctable.encode(str(sentence), maxlen)
for i, sentence in enumerate(parced_prices):
    y[i] = ctable.encode(str(sentence), output_len)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

model = Sequential()
model.add(LSTM(config.hidden_size, input_shape=(maxlen, len(chars))))
model.add(RepeatVector(output_len))
model.add(LSTM(config.hidden_size, return_sequences=True))
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
             metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=config.batch_size,
              epochs=1,
              validation_data=(x_val, y_val),callbacks=[WandbCallback()])
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print('G',guess, end='\n')


Total prices: 3190
Vectorization...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               100864    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 10, 128)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 128)           131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 68)            8772      
Total params: 241,220
Trainable params: 241,220
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 1
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 6s 2ms/sample - loss: 2.9588 - acc: 0.3653 - val_loss:

2871/2871 [==============================] - 4s 1ms/sample - loss: 1.0646 - acc: 0.6390 - val_loss: 1.0501 - val_acc: 0.6502
Q $210,000                       T 210000     ☒ G 199000    
Q $2,850,000                     T 2850000    ☒ G 199000    
Q $199,000                       T 199000     ☑ G 199000    
Q $3,879,900+                    T 3879900    ☒ G 199000    
Q $1,425,000                     T 1425000    ☒ G 199000    
Q $840,000+                      T 840000     ☒ G 199000    
Q $2,695,000                     T 2695000    ☒ G 199000    
Q $1,450,000                     T 1450000    ☒ G 199000    
Q $414,000                       T 414000     ☒ G 199000    
Q Est. $1,846,725                T 1846725    ☒ G 199000    

--------------------------------------------------
Iteration 11
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 4s 1ms/sample - loss: 1.0553 - acc: 0.6472 - val_loss: 1.0439 - val_acc: 0.6502
Q $599,900                  

2871/2871 [==============================] - 4s 1ms/sample - loss: 1.0474 - acc: 0.6491 - val_loss: 1.0412 - val_acc: 0.6502
Q $1,179,000                     T 1179000    ☒ G 199000    
Q $2,229,900+                    T 2229900    ☒ G 199000    
Q $659,000                       T 659000     ☒ G 199000    
Q $3,000,000                     T 3000000    ☒ G 199000    
Q $1,500,000                     T 1500000    ☒ G 199000    
Q $629,000                       T 629000     ☒ G 199000    
Q $2,499,999                     T 2499999    ☒ G 199000    
Q $1,395,000                     T 1395000    ☒ G 199000    
Q $1,022,250 - $2,800,000        T 1022250    ☒ G 199000    
Q $565,000                       T 565000     ☒ G 199000    

--------------------------------------------------
Iteration 21
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 4s 1ms/sample - loss: 1.0479 - acc: 0.6522 - val_loss: 1.0450 - val_acc: 0.6502
Q $685,000                  

2871/2871 [==============================] - 4s 1ms/sample - loss: 1.0431 - acc: 0.6514 - val_loss: 1.0525 - val_acc: 0.6502
Q $800                           T 800        ☒ G 199000    
Q $26,500                        T 26500      ☒ G 199000    
Q $250,000                       T 250000     ☒ G 199000    
Q $682,000                       T 682000     ☒ G 199000    
Q $800                           T 800        ☒ G 199000    
Q $299,000 - $2,134,000          T 299000     ☒ G 199000    
Q $1,100,000                     T 1100000    ☒ G 199000    
Q $300,000                       T 300000     ☒ G 199000    
Q $949,000                       T 949000     ☒ G 199000    
Q $3,000,000                     T 3000000    ☒ G 199000    

--------------------------------------------------
Iteration 31
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 4s 1ms/sample - loss: 1.0435 - acc: 0.6526 - val_loss: 1.0504 - val_acc: 0.6502
Q $1,350,000                

2871/2871 [==============================] - 4s 1ms/sample - loss: 1.0156 - acc: 0.6561 - val_loss: 1.0501 - val_acc: 0.6530
Q $1,895,000                     T 1895000    ☒ G 190000    
Q $40,000                        T 40000      ☒ G 190000    
Q $1,025,000                     T 1025000    ☒ G 190000    
Q $55,000                        T 55000      ☒ G 190000    
Q $700,000                       T 700000     ☒ G 190000    
Q $1,450,000                     T 1450000    ☒ G 190000    
Q $850,000                       T 850000     ☒ G 190000    
Q $465,000                       T 465000     ☒ G 190000    
Q $1,120,000                     T 1120000    ☒ G 190000    
Q $250,000                       T 250000     ☒ G 190000    

--------------------------------------------------
Iteration 41
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 4s 1ms/sample - loss: 1.0131 - acc: 0.6605 - val_loss: 1.0200 - val_acc: 0.6545
Q $450,000                  

...

2871/2871 [==============================] - 4s 1ms/sample - loss: 0.0351 - acc: 0.9957 - val_loss: 0.0455 - val_acc: 0.9931
Q $1,615,000                     T 1615000    ☑ G 1615000   
Q $312,999                       T 312999     ☑ G 312999    
Q $40,000                        T 40000      ☑ G 40000     
Q $529,154                       T 529154     ☑ G 529154    
Q $999,999                       T 999999     ☑ G 999999    
Q $240,000                       T 240000     ☑ G 240000    
Q $900,000                       T 900000     ☑ G 900000    
Q $339,900                       T 339900     ☑ G 339900    
Q $240,000                       T 240000     ☑ G 240000    
Q $595,000                       T 595000     ☑ G 595000    

--------------------------------------------------
Iteration 151
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 4s 1ms/sample - loss: 0.0309 - acc: 0.9965 - val_loss: 0.0417 - val_acc: 0.9950
Q Est. $1,251,730          

2871/2871 [==============================] - 4s 1ms/sample - loss: 0.0184 - acc: 0.9993 - val_loss: 0.0282 - val_acc: 0.9975
Q $32,000                        T 32000      ☑ G 32000     
Q $350,000                       T 350000     ☑ G 350000    
Q $104,900                       T 104900     ☑ G 104900    
Q $7,500,000                     T 7500000    ☑ G 7500000   
Q $2,370,755                     T 2370755    ☑ G 2370755   
Q $192,500                       T 192500     ☑ G 192500    
Q $1,000,000                     T 1000000    ☑ G 1000000   
Q $1,775,000                     T 1775000    ☑ G 1775000   
Q $999,000                       T 999000     ☑ G 999000    
Q $4,875,000                     T 4875000    ☑ G 4875000   

--------------------------------------------------
Iteration 161
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 4s 1ms/sample - loss: 0.0182 - acc: 0.9990 - val_loss: 0.0277 - val_acc: 0.9981
Q $649,000                 

2871/2871 [==============================] - 4s 1ms/sample - loss: 0.5252 - acc: 0.8384 - val_loss: 0.3292 - val_acc: 0.8928
Q $811,065                       T 811065     ☒ G 81140     
Q $685,000                       T 685000     ☒ G 880000    
Q $159,999                       T 159999     ☒ G 199999    
Q $7,500                         T 7500       ☒ G 7700      
Q $450,000                       T 450000     ☑ G 450000    
Q $1,500,000                     T 1500000    ☑ G 1500000   
Q $465,000                       T 465000     ☑ G 465000    
Q $13,625,000                    T 13625000   ☒ G 1333500   
Q $58,000                        T 58000      ☒ G 89000     
Q $99,000                        T 99000      ☑ G 99000     

--------------------------------------------------
Iteration 171
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 4s 1ms/sample - loss: 0.2522 - acc: 0.9208 - val_loss: 0.2090 - val_acc: 0.9326
Q $210,000                 

2871/2871 [==============================] - 4s 1ms/sample - loss: 0.0441 - acc: 0.9915 - val_loss: 0.0525 - val_acc: 0.9878
Q $465,000                       T 465000     ☑ G 465000    
Q $659,000                       T 659000     ☑ G 659000    
Q $224,900                       T 224900     ☑ G 224900    
Q $2,995,000                     T 2995000    ☑ G 2995000   
Q $1,000,000                     T 1000000    ☑ G 1000000   
Q $1,500,000                     T 1500000    ☑ G 1500000   
Q $1,750,000                     T 1750000    ☑ G 1750000   
Q $1,300,000                     T 1300000    ☑ G 1300000   
Q $547,000                       T 547000     ☑ G 547000    
Q $898,000                       T 898000     ☑ G 898000    

--------------------------------------------------
Iteration 181
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 4s 1ms/sample - loss: 0.0401 - acc: 0.9931 - val_loss: 0.0461 - val_acc: 0.9925
Q $22,000                  

2871/2871 [==============================] - 4s 1ms/sample - loss: 0.0215 - acc: 0.9993 - val_loss: 0.0268 - val_acc: 0.9987
Q $1,179,000                     T 1179000    ☑ G 1179000   
Q $629,000                       T 629000     ☑ G 629000    
Q $312,999                       T 312999     ☑ G 312999    
Q $189,900                       T 189900     ☑ G 189900    
Q $749,000                       T 749000     ☑ G 749000    
Q $898,000                       T 898000     ☑ G 898000    
Q $745,000                       T 745000     ☑ G 745000    
Q $1,450,000                     T 1450000    ☑ G 1450000   
Q $2,695,000                     T 2695000    ☑ G 2695000   
Q $1,500,000                     T 1500000    ☑ G 1500000   

--------------------------------------------------
Iteration 191
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 4s 1ms/sample - loss: 0.0194 - acc: 0.9997 - val_loss: 0.0263 - val_acc: 0.9978
Q $372,900+                

In [5]:
for iteration in range(1, 50):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=config.batch_size,
              epochs=1,
              validation_data=(x_val, y_val),callbacks=[WandbCallback()])
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print('G',guess, end='\n')



--------------------------------------------------
Iteration 1
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 6s 2ms/sample - loss: 0.0121 - acc: 0.9999 - val_loss: 0.0170 - val_acc: 0.9981
Q $199,000                       T 199000     ☑ G 199000    
Q $350,000                       T 350000     ☑ G 350000    
Q $2,495,000                     T 2495000    ☑ G 2495000   
Q $2,275,000                     T 2275000    ☑ G 2275000   
Q $3,200,000                     T 3200000    ☑ G 3200000   
Q $1,175,000                     T 1175000    ☑ G 1175000   
Q $3,784,995+                    T 3784995    ☑ G 3784995   
Q Est. $73,822                   T 73822      ☑ G 73822     
Q $650,000                       T 650000     ☑ G 650000    
Q $205,000                       T 205000     ☑ G 205000    

--------------------------------------------------
Iteration 2
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] 

2871/2871 [==============================] - 4s 2ms/sample - loss: 0.0078 - acc: 0.9999 - val_loss: 0.0107 - val_acc: 0.9991
Q $26,500                        T 26500      ☑ G 26500     
Q $595,000                       T 595000     ☑ G 595000    
Q $649,900                       T 649900     ☑ G 649900    
Q $25,900,000                    T 25900000   ☑ G 25900000  
Q $1,179,000                     T 1179000    ☑ G 1179000   
Q $240,000                       T 240000     ☑ G 240000    
Q Est. $1,251,730                T 1251730    ☒ G 1151730   
Q $192,500                       T 192500     ☑ G 192500    
Q $299,000                       T 299000     ☑ G 299000    
Q $224,900                       T 224900     ☑ G 224900    

--------------------------------------------------
Iteration 12
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 4s 2ms/sample - loss: 0.0075 - acc: 1.0000 - val_loss: 0.0102 - val_acc: 0.9991
Q $2,745,000                

2871/2871 [==============================] - 4s 2ms/sample - loss: 0.0054 - acc: 1.0000 - val_loss: 0.0076 - val_acc: 1.0000
Q $1,075,000                     T 1075000    ☑ G 1075000   
Q $3,000,000                     T 3000000    ☑ G 3000000   
Q $60,000 - $388,500             T 60000      ☑ G 60000     
Q $650,000                       T 650000     ☑ G 650000    
Q $1,385,000                     T 1385000    ☑ G 1385000   
Q $60,000 - $388,500             T 60000      ☑ G 60000     
Q $540,000                       T 540000     ☑ G 540000    
Q $98,000                        T 98000      ☑ G 98000     
Q $19,900                        T 19900      ☑ G 19900     
Q $1,999,999                     T 1999999    ☑ G 1999999   

--------------------------------------------------
Iteration 22
Train on 2871 samples, validate on 319 samples
2871/2871 [==============================] - 5s 2ms/sample - loss: 0.0053 - acc: 1.0000 - val_loss: 0.0074 - val_acc: 1.0000
Q $685,000                  

KeyboardInterrupt: 